In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
import pandas as pd
import numpy as np
import time
from preprocessing import *
from JREP import JREP

# set_seed(42)

In [2]:
df = pd.read_csv(r'..\output_csv\ANL-Intrepid-2009-1.swf.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68936 entries, 0 to 68935
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    68936 non-null  float64
 1   submit_time               68936 non-null  float64
 2   wait_time                 68936 non-null  float64
 3   run_time                  68936 non-null  float64
 4   num_allocated_processors  68936 non-null  float64
 5   avg_cpu_time_used         68936 non-null  float64
 6   used_memory               68936 non-null  float64
 7   requested_processors      68936 non-null  float64
 8   requested_time            68936 non-null  float64
 9   requested_memory          68936 non-null  float64
 10  status                    68936 non-null  float64
 11  user_id                   68936 non-null  float64
 12  group_id                  68936 non-null  float64
 13  executable_id             68936 non-null  float64
 14  queue_

In [3]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,1.0,0.0,6680.0,7560.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
1,2.0,7.0,14297.0,7568.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,3.0,1590.0,17322.0,7561.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
3,4.0,2205.0,61.0,14972.0,8192.0,-1.0,-1.0,8192.0,21600.0,-1.0,-1.0,2.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0
4,5.0,2566.0,17357.0,7571.0,2048.0,-1.0,-1.0,2048.0,10800.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0


In [4]:
feature_columns = ['requested_processors', 'requested_time', 'submit_time', 'wait_time', 'user_id', 'queue_id']
target_column = 'run_time'

X_train, X_test, Y_train, Y_test, scaler = prepare_data(df, feature_columns, target_column, statuss=-1)

In [5]:
rmse_lst, mae_lst, mse_lst, r2_lst, infer_time_lst = [], [], [], [], []
for i in range(10):
    print(f"Iteration {i+1}/100")
    JREP_model = JREP()
    JREP_model.fit(X_train, Y_train)

    start = time.time()
    Y_pred = JREP_model.predict(X_test, scaler, len(feature_columns))
    end = time.time()
    rmse, mae, mse, r2 = JREP_model.evaluate_model(Y_test, Y_pred, scaler, len(feature_columns))

    rmse_lst.append(rmse)
    mae_lst.append(mae)
    mse_lst.append(mse)
    r2_lst.append(r2)
    infer_time_lst.append((end - start) / len(Y_pred))

    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R2: {r2}")
    print(f'Inference Time: {(end - start)/len(Y_pred)} seconds')

# Save results to CSV
results_df = pd.DataFrame({
    'RMSE': rmse_lst,
    'MAE': mae_lst,
    'MSE': mse_lst,
    'R2': r2_lst,
    'Inference Time': infer_time_lst
})
results_df.to_csv('../output_ANL/JREP_results.csv', index=False)

Iteration 1/100
RMSE: 6801.196178838301
MAE: 1411.962093263012
MSE: 46256269.46304472
R2: 0.6115159508694251
Inference Time: 0.0018152803247556465 seconds
Iteration 2/100
RMSE: 6399.070695252421
MAE: 1394.1602510246796
MSE: 40948105.7628383
R2: 0.6560966520725695
Inference Time: 0.001977191364408508 seconds
Iteration 3/100
RMSE: 6688.976414996574
MAE: 1456.432478840192
MSE: 44742405.48038042
R2: 0.6242301627296838
Inference Time: 0.0014870753967575396 seconds
Iteration 4/100
RMSE: 6601.009801014829
MAE: 1342.9082590959285
MSE: 43573330.39309383
R2: 0.6340486593122847
Inference Time: 0.0014544196806261982 seconds
Iteration 5/100
RMSE: 6481.926771140256
MAE: 1364.1016360318936
MSE: 42015374.66642475
R2: 0.6471331764185793
Inference Time: 0.0027606311306425293 seconds
Iteration 6/100
RMSE: 6588.04807609822
MAE: 1284.9761804850755
MSE: 43402377.45298146
R2: 0.6354844104257353
Inference Time: 0.0013863317371713825 seconds
Iteration 7/100
RMSE: 6554.330212720268
MAE: 1338.1123189139853
MSE: 